In [1]:
import json
import pandas as pd
from datetime import datetime, timezone

## First look at the structure of the bookmarks.json file

In [ ]:
# Path to the bookmarks.json file
file_path = 'example_bookmarks.json'

# Open file and load JSON data
with open(file_path, 'r', encoding='utf-8') as file:
    bookmarks_data = json.load(file)

# Display JSON data
print(json.dumps(bookmarks_data, indent=4, ensure_ascii=False))

## Editing the bookmarks.json

In [ ]:
# Function for converting microseconds (Unix epoch) to datetime
def convert_timestamp(timestamp):
    # Convert microseconds to seconds
    timestamp_in_seconds = timestamp / 1_000_000
    # Create a datetime object with UTC time zone
    return datetime.fromtimestamp(timestamp_in_seconds, tz=timezone.utc)

# Function for recursive extraction of bookmarks, including the folder structure
def extract_bookmarks_with_folder(bookmark_data, folder_path=''):
    bookmarks_list = []
    if isinstance(bookmark_data, list):
        # If the data is a list (folder or bookmark)
        for bookmark in bookmark_data:
            bookmarks_list.extend(extract_bookmarks_with_folder(bookmark, folder_path))  # Recursion
    elif isinstance(bookmark_data, dict):
        # If it is a dictionary (e.g. a folder or bookmark)
        if 'uri' in bookmark_data:  # If bookmarks have 'uri' and 'title'
            # Add the full folder path to the bookmark
            full_folder_path = folder_path.strip(' > ')  # Removing the last " > "
            # Converting the timestamp to DateTime
            date_added = convert_timestamp(bookmark_data.get('dateAdded', 0))
            last_modified = convert_timestamp(bookmark_data.get('lastModified', 0))
            # Include all desired fields in the dictionary
            bookmarks_list.append({
                'Title': bookmark_data['title'],
                'URL': bookmark_data['uri'],
                'Icon': bookmark_data.get('iconUri', ''),  # Optional icon field
                'Date Added': date_added,
                'Last Modified': last_modified,
                'Folder Path': full_folder_path  # The folder path
            })
        # If there are 'children', continue recursively through the subfolders
        if 'children' in bookmark_data:
            # Add the current folder to the folder structure and continue recursively
            new_folder_path = f"{folder_path} > {bookmark_data['title']}" if bookmark_data['title'] else folder_path
            bookmarks_list.extend(extract_bookmarks_with_folder(bookmark_data['children'], new_folder_path))
    return bookmarks_list

# Path to the JSON file
file_path = 'example_bookmarks.json'

# Open file and load JSON data
with open(file_path, 'r', encoding='utf-8') as file:
    bookmarks_data = json.load(file)

# Recursive extraction of bookmarks with folder path
bookmarks_list = extract_bookmarks_with_folder(bookmarks_data)

# Check whether bookmarks have been extracted
if bookmarks_list:
    # Create the DataFrame
    df = pd.DataFrame(bookmarks_list)
    print(f'DataFrame erfolgreich erstellt!')
else:
    print("Keine Lesezeichen gefunden!")

In [ ]:
# Show created DataFrame
df

In [5]:
# Remove the time zone information from the date values (only for Excel)
df['Date Added'] = df['Date Added'].dt.tz_localize(None)
df['Last Modified'] = df['Last Modified'].dt.tz_localize(None)

## Save DataFrame in an Excel file

In [ ]:
# Export the DataFrame to Excel
output_file_path = 'example_bookmarks.xlsx'
df.to_excel(output_file_path, index=False)  # index=False to not save the index column

if output_file_path:
    print(f'DataFrame erfolgreich nach {output_file_path} exportiert!')
else:
    print("Keine Lesezeichen gefunden!")